## Feature Extaction Test

In [1]:
import sys
sys.path.insert(0, '..')
from metrics.metrics_utils import get_tp_fp_fn_center_patch_criteria
from general_utils.plots import plot_bboxes_over_image, simple_im_show, simple_im_show2
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from mc_candidate_proposal.morphology_mc import MorphologyCalcificationDetection
from feature_extraction.feature_extraction import CandidatesFeatureExtraction, CandidatesFeatureExtraction_MP
from database.dataset import *
import feature_extraction.haar_features.haar_modules as hm

import pickle

### Database initialization

In [2]:
db = INBreast_Dataset(
    return_lesions_mask=True,
    level='image',
    max_lesion_diam_mm=None,
    extract_patches=False,
    partitions = ['train', 'validation'],
    lesion_types = ['calcification', 'cluster'],
    cropped_imgs = True,
    keep_just_images_of_lesion_type = False,
    use_muscle_mask=True,
    ignore_diameter_px = 15
)

### Detectors Parameters and initialization

In [3]:
hd = HoughCalcificationDetection()
CENTER_CROP_PATCH = 14
PATCH_SIZE = 14

GABOR_PARAMS = {'scale': 2, 'orientation': 3,
                'max_freq': 0.2, 'ksize': (20, 20), 'sigma': 3}
WAVELET_PARAMS = {'angles': [0, np.pi/4, np.pi/2]}
# HAAR_PARAMS = {
#     'skimage': {
#         'feature_type': [], 'feature_coord': []
#     },
#     'ours': {
#         'horizontal_feature_types': [(hm.Feature3h3v, 3, 3)],
#         'rotated_feature_types': None,
#         'horizontal_feature_selection': None,
#         'rotated_feature_selection': None,
#     },
#     'patch_size': PATCH_SIZE
# }


# # IS THIS FINAL HAAR? MAYBE
# path = (Path.cwd().parent / f'feature_extraction/haar_features/')
# with open(path/'final_feat_selection.p', 'rb') as f:
#     selection = pickle.load(f)
    
# HAAR_PARAMS = {
#     'skimage': {
#         'feature_type': selection['skimage_haar_feature_types_sel'],
#         'feature_coord': selection['skimage_haar_feature_coords_sel']
#     },
#     'ours': {
#         'horizontal_feature_selection': selection['hor_feats_selection'].tolist(),
#         'rotated_feature_selection': selection['rot_feats_selection'].tolist(),
#         'rotated_feature_types': None,
#         'horizontal_feature_types': None
#     },
#     'patch_size': 14
# }

cfe = CandidatesFeatureExtraction_MP(patch_size=PATCH_SIZE, fos=True, gabor_params=GABOR_PARAMS,
                                  wavelet_params=WAVELET_PARAMS, haar_params=None, n_jobs=4)

### Detector initialization

In [4]:
db_range = list(range(len(db)))

## FE

### Hough

In [6]:
with open('/home/vzalevskyi/projects/calc-det/data/final_haar_500_feat_selection_hough.p', 'rb') as f:
    selection = pickle.load(f)
    
HAAR_PARAMS = {
    'skimage': {
        'feature_type': selection['skimage_haar_feature_types_sel'],
        'feature_coord': selection['skimage_haar_feature_coords_sel']
    },
    'ours': {
        'horizontal_feature_selection': selection['hor_feats_selection'].tolist(),
        'rotated_feature_selection': selection['rot_feats_selection'].tolist(),
        'rotated_feature_types': None,
        'horizontal_feature_types': None
    },
    'patch_size': 14
}

cfe = CandidatesFeatureExtraction_MP(patch_size=PATCH_SIZE, fos=False, gabor_params=None,
                                  wavelet_params=None, haar_params=HAAR_PARAMS, n_jobs=4)

In [7]:
data_path = Path.cwd().parent.parent/'data/features/hough'

fdf = []
fns_df = []
ingnored_df = []

batch = 1

for idx in tqdm(db_range[:110]):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']
    muscle_mask = db_sample['muscle_mask']

    # candidate selection
    candidates = hd.detect(image, image_id, load_processed_images=True, hough2=False, muscle_mask=muscle_mask)
     # labeling of candidates:
    tp, fp, fn, ignored_candidates = get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, None, 14)
    
    candidates = pd.concat([tp, fp], axis=0, ignore_index=True)
    
    # Extracting features
    labels = np.where(candidates.label.values == 'TP', True, False)
    
    X = candidates.drop_duplicates(subset='repeted_idxs')
    X = cfe.extract_features(X.loc[:, ['x','y','radius']].values.astype(int), image)
    
    # Get features dfs
    X = pd.DataFrame(data=X, columns=cfe.feature_names)
    X.index = candidates.drop_duplicates(subset='repeted_idxs').index

    res = X.loc[candidates.repeted_idxs.tolist(),:]
    res['img_id'] = image_id
    res['repeted_idxs'] = candidates.repeted_idxs.tolist()
    res['matching_gt'] = candidates.matching_gt.tolist()
    res['label'] = labels

    # Generate a fn dataframe to compute frocs
    fn['img_id'] = image_id
    fns_df.append(fn)
    fdf.append(res)
    ingnored_df.append(ignored_candidates)
    
all_data_df = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)
ingnored_df = pd.concat(ingnored_df, ignore_index=True)

all_data_df.to_feather(str(data_path/f'all_data_df_{batch}_haar.f'))
fns_df.to_feather(str(data_path/f'fns_df_{batch}_haar.f'))
ingnored_df.to_feather(str(data_path/f'ingnored_df_{batch}_haar.f'))

100%|██████████| 110/110 [16:49<00:00,  9.18s/it]


In [8]:
del all_data_df
del fns_df
del fdf
del ingnored_df

In [9]:
data_path = Path.cwd().parent.parent/'data/features/hough'

fdf = []
fns_df = []
ingnored_df = []

batch = 2

for idx in tqdm(db_range[110:]):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']
    muscle_mask = db_sample['muscle_mask']

    # candidate selection
    candidates = hd.detect(image, image_id, load_processed_images=True, hough2=False, muscle_mask=muscle_mask)
     # labeling of candidates:
    tp, fp, fn, ignored_candidates = get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, None, 14)
    
    candidates = pd.concat([tp, fp], axis=0, ignore_index=True)
    
    # Extracting features
    labels = np.where(candidates.label.values == 'TP', True, False)
    
    X = candidates.drop_duplicates(subset='repeted_idxs')
    X = cfe.extract_features(X.loc[:, ['x','y','radius']].values.astype(int), image)
    
    # Get features dfs
    X = pd.DataFrame(data=X, columns=cfe.feature_names)
    X.index = candidates.drop_duplicates(subset='repeted_idxs').index

    res = X.loc[candidates.repeted_idxs.tolist(),:]
    res['img_id'] = image_id
    res['repeted_idxs'] = candidates.repeted_idxs.tolist()
    res['matching_gt'] = candidates.matching_gt.tolist()
    res['label'] = labels

    # Generate a fn dataframe to compute frocs
    fn['img_id'] = image_id
    fns_df.append(fn)
    fdf.append(res)
    ingnored_df.append(ignored_candidates)
    
all_data_df = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)
ingnored_df = pd.concat(ingnored_df, ignore_index=True)

all_data_df.to_feather(str(data_path/f'all_data_df_{batch}_haar.f'))
fns_df.to_feather(str(data_path/f'fns_df_{batch}_haar.f'))
ingnored_df.to_feather(str(data_path/f'ingnored_df_{batch}_haar.f'))

100%|██████████| 100/100 [15:42<00:00,  9.43s/it]


In [10]:
val_img_ids = set(db.df.img_id[db.df.partition == 'validation'].unique())
normal_img_ids = set(db.get_normal_imgs_ids())

validation_mask = all_data_df.img_id.isin(val_img_ids)
validation_normals_mask = all_data_df.img_id.isin(normal_img_ids&val_img_ids)
normals_mask = all_data_df.img_id.isin(normal_img_ids)

print(f'General train+val sensitivity: {all_data_df.label.sum()/(all_data_df.label.sum() + len(fns_df))}')
print(f'General val sensitivity: {all_data_df[validation_mask].label.sum()/(all_data_df[validation_mask].label.sum() + len(fns_df[fns_df.img_id.isin(val_img_ids)]))}')

General train+val sensitivity: 0.8771626297577855
General val sensitivity: 0.9083333333333333


### Morphology

In [5]:
threshold = 0.95
min_distance = 6
area = 14
rbd_p = '/home/vzalevskyi/projects/calc-det/data/recounstructed_by_dialation_img'
md = MorphologyCalcificationDetection(rbd_p, threshold, min_distance, area*area, filter_muscle_region=True, store_intermediate=False)

In [7]:
CENTER_CROP_PATCH = 14
PATCH_SIZE = 14

GABOR_PARAMS = {'scale': 2, 'orientation': 3,
                'max_freq': 0.2, 'ksize': (20, 20), 'sigma': 3}
WAVELET_PARAMS = {'angles': [0, np.pi/4, np.pi/2]}



with open('/home/vzalevskyi/projects/calc-det/data/final_haar_500_feat_selection_gsm.p', 'rb') as f:
    selection = pickle.load(f)
    
HAAR_PARAMS = {
    'skimage': {
        'feature_type': selection['skimage_haar_feature_types_sel'],
        'feature_coord': selection['skimage_haar_feature_coords_sel']
    },
    'ours': {
        'horizontal_feature_selection': selection['hor_feats_selection'].tolist(),
        'rotated_feature_selection': selection['rot_feats_selection'].tolist(),
        'rotated_feature_types': None,
        'horizontal_feature_types': None
    },
    'patch_size': 14
}

cfe = CandidatesFeatureExtraction_MP(patch_size=PATCH_SIZE, fos=True, gabor_params=GABOR_PARAMS,
                                  wavelet_params=WAVELET_PARAMS, haar_params=HAAR_PARAMS, n_jobs=4)

In [8]:
data_path = Path.cwd().parent.parent/'data/features/morph'

fdf = []
fns_df = []
ingnored_df = []

batch = 1

for idx in tqdm(db_range[:]):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']
    muscle_mask = db_sample['muscle_mask']

    # candidate selection
    candidates = md.detect(image, image_id, muscle_mask=muscle_mask)
     # labeling of candidates:
    tp, fp, fn, ignored_candidates = get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, None, 14)
    
    candidates = pd.concat([tp, fp], axis=0, ignore_index=True)
    
    # Extracting features
    labels = np.where(candidates.label.values == 'TP', True, False)
    
    X = candidates.drop_duplicates(subset='repeted_idxs')
    X = cfe.extract_features(X.loc[:, ['x','y','radius']].values.astype(int), image)
    
    # Get features dfs
    X = pd.DataFrame(data=X, columns=cfe.feature_names)
    X.index = candidates.drop_duplicates(subset='repeted_idxs').index

    res = X.loc[candidates.repeted_idxs.tolist(),:]
    res['img_id'] = image_id
    res['repeted_idxs'] = candidates.repeted_idxs.tolist()
    res['matching_gt'] = candidates.matching_gt.tolist()
    res['label'] = labels

    # Generate a fn dataframe to compute frocs
    fn['img_id'] = image_id
    fns_df.append(fn)
    fdf.append(res)
    ingnored_df.append(ignored_candidates)
    
all_data_df = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)
ingnored_df = pd.concat(ingnored_df, ignore_index=True)

all_data_df.to_feather(str(data_path/f'all_data_df_{batch}.f'))
fns_df.to_feather(str(data_path/f'fns_df_{batch}.f'))
ingnored_df.to_feather(str(data_path/f'ingnored_df_{batch}.f'))

100%|██████████| 210/210 [2:25:38<00:00, 41.61s/it]  


In [16]:
val_img_ids = set(db.df.img_id[db.df.partition == 'validation'].unique())
normal_img_ids = set(db.get_normal_imgs_ids())

validation_mask = all_data_df.img_id.isin(val_img_ids)
validation_normals_mask = all_data_df.img_id.isin(normal_img_ids&val_img_ids)
normals_mask = all_data_df.img_id.isin(normal_img_ids)

print(f'General train+val sensitivity: {all_data_df.label.sum()/(all_data_df.label.sum() + len(fns_df))}')
print(f'General val sensitivity: {all_data_df[validation_mask].label.sum()/(all_data_df[validation_mask].label.sum() + len(fns_df[fns_df.img_id.isin(val_img_ids)]))}')

General train+val sensitivity: 0.8089813800657174
General val sensitivity: 0.7985611510791367


## Merging batches

Merging differnet batches of features

In [3]:
features_data1 = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/final_features_hough1.f')
features_data2 = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/final_features_hough2.f')

features_data = pd.concat([features_data1, features_data2])

del features_data1
del features_data2

features_data.reset_index().drop(columns=['index']).to_feather('/home/vzalevskyi/projects/data/features/hough/features_hough.f')


In [7]:
fnsdf1 =  pd.read_feather('/home/vzalevskyi/projects/data/features/hough/fns_df_1.f')
fnsdf2 =  pd.read_feather('/home/vzalevskyi/projects/data/features/hough/fns_df_2.f')

ignored1 = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/ingnored_df_1.f')
ignored2 = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/ingnored_df_2.f')

fnsdf = pd.concat([fnsdf1, fnsdf2]).reset_index().drop(columns=['index'])
ignored = pd.concat([ignored1, ignored2]).reset_index().drop(columns=['index'])

fnsdf.to_feather('/home/vzalevskyi/projects/data/features/hough/fnsdf_hough.f')
ignored.to_feather('/home/vzalevskyi/projects/data/features/hough/ignored_hough.f')

Merging haar features to rest of features

In [6]:
# used to merge batches of features, ignore
# features_data1 = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/all_data_df_2.f')
# features_data1_haar = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/all_data_df_2_haar.f')


# merged_data = features_data1.merge(features_data1_haar, on=['img_id', 'repeted_idxs', 'matching_gt'], how='inner', suffixes=(None, '_y'), validate='one_to_one')
# merged_data = merged_data.drop(columns=[x for x in merged_data.columns if '_y' in x])

# merged_data.to_feather('/home/vzalevskyi/projects/data/features/hough/final_features_hough2.f')

# fns_df_rest = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/fns_df_1.f')
# fns_df_haar = pd.read_feather('/home/vzalevskyi/projects/data/features/hough/fns_df_1_haar.f')

# fns_df_rest == fns_df_haar